# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=103

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-11_15-39-55_18336/logs.
Waiting for redis server at 127.0.0.1:43604 to respond...
Waiting for redis server at 127.0.0.1:54079 to respond...
Starting the Plasma object store with 6.5546633210000005 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=30f0302ce2f7f65d3a1489cfbc57b40dfb05c4335bfcd93a



{'node_ip_address': '172.16.123.117',
 'object_store_addresses': ['/tmp/ray/session_2019-03-11_15-39-55_18336/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-03-11_15-39-55_18336/sockets/raylet'],
 'redis_address': '172.16.123.117:43604',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=30f0302ce2f7f65d3a1489cfbc57b40dfb05c4335bfcd93a'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [64, 32]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy(), 'rl_1': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return agent_id

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn)
        }
    })

 Starting SUMO on port 58479


23.849015537022204
25.000549626928986


In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.3/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv-v0_0_2019-03-11_15-39-58ssrs95mm -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_15-40-41
  done: false
  episode_len_mean: 103.0
  episode_reward_max: 20.473525119472047
  episode_reward_mean: 10.040975470023351
  episode_reward_min: 4.446865024276959
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 2139.944
    load_time_ms: 150.256
    num_steps_sampled: 2200
    num_steps_trained: 

  custom_metrics: {}
  date: 2019-03-11_15-41-10
  done: false
  episode_len_mean: 103.0
  episode_reward_max: 33.8636255653574
  episode_reward_mean: 10.943074297584197
  episode_reward_min: 2.5088519950107915
  episodes_this_iter: 22
  episodes_total: 106
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1142.806
    load_time_ms: 32.316
    num_steps_sampled: 11000
    num_steps_trained: 11000
    rl_0:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4244626760482788
      kl: 0.0010611612815409899
      policy_loss: -0.003908729180693626
      total_loss: 3.588604211807251
      vf_explained_var: 0.33883801102638245
      vf_loss: 3.5924999713897705
    rl_1:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4246826171875
      kl: 0.0010549579747021198
      policy_loss: -0.005975959822535515
      total_loss: 3.2014319896698
      vf_explained_va

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 70 s, 9 iter, 19800 ts, 14.6 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_15-41-50
  done: false
  episode_len_mean: 103.0
  episode_reward_max: 47.49019103099962
  episode_reward_mean: 15.807334070455195
  episode_reward_min: 2.4599815605381488
  episodes_this_iter: 20
  episodes_total: 212
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1094.976
    load_time_ms: 17.513
    num_steps_sampled: 22000
    num_steps_trained: 22000
    rl_0:
      cur_kl_coeff: 0.0003906250058207661
      cur_lr: 4.999999873689376e-05
      entropy: 1.4063326120376587
      kl: 0.0051465872675180435
      policy_loss: -0.007830937393009663
      total_lo

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.2/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 129 s, 14 iter, 30800 ts, 12.6 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_15-42-49
  done: false
  episode_len_mean: 102.44
  episode_reward_max: 55.49541427057067
  episode_reward_mean: 9.63875273597274
  episode_reward_min: -164.71235867701722
  episodes_this_iter: 22
  episodes_total: 320
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1274.816
    load_time_ms: 2.974
    num_steps_sampled: 33000
    num_steps_trained: 33000
    rl_0:
      cur_kl_coeff: 1.220703143189894e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.4215896129608154
      kl: 0.001090708072297275
      policy_loss: -0.0007822535699233413
      total_l

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 167 s, 19 iter, 41800 ts, 17.3 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_15-43-24
  done: false
  episode_len_mean: 102.74
  episode_reward_max: 62.18640583172605
  episode_reward_mean: 16.172823571421098
  episode_reward_min: -160.54216454788255
  episodes_this_iter: 21
  episodes_total: 427
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1246.459
    load_time_ms: 2.864
    num_steps_sampled: 44000
    num_steps_trained: 44000
    rl_0:
      cur_kl_coeff: 3.814697322468419e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.4268198013305664
      kl: 0.001855549169704318
      policy_loss: -0.002835640450939536
      total_

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 3123 s, 394 iter, 866800 ts, 125 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_16-33-05
  done: false
  episode_len_mean: 91.74
  episode_reward_max: 240.19705239231513
  episode_reward_mean: 132.06457794625007
  episode_reward_min: -167.3688269562991
  episodes_this_iter: 23
  episodes_total: 8936
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 891.973
    load_time_ms: 2.773
    num_steps_sampled: 869000
    num_steps_trained: 869000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.4236995577812195
      kl: 0.0174117274582386
      policy_loss: 0.0061396388337016106
      total_loss: 95.37989044

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.2/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 3167 s, 399 iter, 877800 ts, 130 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_16-33-48
  done: false
  episode_len_mean: 89.39
  episode_reward_max: 241.48477796531748
  episode_reward_mean: 111.8696304760306
  episode_reward_min: -165.56951864422348
  episodes_this_iter: 26
  episodes_total: 9058
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1133.965
    load_time_ms: 3.455
    num_steps_sampled: 880000
    num_steps_trained: 880000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.46357831358909607
      kl: 0.024036461487412453
      policy_loss: -0.0002579549909569323
      total_loss: 579.73

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 3208 s, 404 iter, 888800 ts, 126 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_16-34-37
  done: false
  episode_len_mean: 92.33
  episode_reward_max: 236.18984714643446
  episode_reward_mean: 136.66983521867908
  episode_reward_min: -160.07673134706494
  episodes_this_iter: 23
  episodes_total: 9178
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1290.668
    load_time_ms: 4.109
    num_steps_sampled: 891000
    num_steps_trained: 891000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.5055211186408997
      kl: 0.006657027639448643
      policy_loss: -6.234821285033831e-06
      total_loss: 326.49

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.9/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 3260 s, 409 iter, 899800 ts, 125 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_16-35-22
  done: false
  episode_len_mean: 91.49
  episode_reward_max: 239.00696989224892
  episode_reward_mean: 125.3952536291121
  episode_reward_min: -155.9125395968146
  episodes_this_iter: 23
  episodes_total: 9297
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1185.497
    load_time_ms: 3.568
    num_steps_sampled: 902000
    num_steps_trained: 902000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.81154865026474
      kl: 0.009831276722252369
      policy_loss: 0.0026027506683021784
      total_loss: 108.4790725

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18379], 3329 s, 414 iter, 910800 ts, 144 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-11_16-36-33
  done: false
  episode_len_mean: 93.86
  episode_reward_max: 241.04244136447002
  episode_reward_mean: 143.94228401746375
  episode_reward_min: -158.56069366239578
  episodes_this_iter: 23
  episodes_total: 9415
  experiment_id: 4ce6132bd9ab404c9dd2064472fe6431
  hostname: Gandalf
  info:
    grad_time_ms: 1592.982
    load_time_ms: 3.77
    num_steps_sampled: 913000
    num_steps_trained: 913000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.673224151134491
      kl: 0.00439930334687233
      policy_loss: -0.003641101298853755
      total_loss: 539.217834